In [1]:
# Import libraries
import requests, os
import urllib.request
import time
from tqdm import tqdm
from bs4 import BeautifulSoup
import statistics
import json


In [2]:
# Set the URL you want to webscrape from
all_games = []
for i in tqdm(range(1,20)):

    # Connect to the URL
    response = requests.get('https://boardgamegeek.com/browse/boardgame/page/{i}')

    # Parse HTML and save to BeautifulSoup object¶
    soup = BeautifulSoup(response.text, "html.parser")

    for row in soup.find_all(id='row_'):
        all_games.append(row.find_all('a')[1].get('href'))
        #print(row.find_all('a')[1].get('href'))


100%|██████████| 19/19 [00:04<00:00,  3.91it/s]


In [3]:
game_id = list(map(lambda x: x.split('/')[2], all_games))

In [4]:
len(game_id)

1900

In [5]:
kid_games = []
for i in tqdm(range(1,15)):

    # Connect to the URL
    response = requests.get(f'https://boardgamegeek.com/childrensgames/browse/boardgame/page/{i}')

    # Parse HTML and save to BeautifulSoup object¶
    soup = BeautifulSoup(response.text, "html.parser")
    for row in soup.find_all(id='row_'):
        if row.find_all('td')[3].contents[0].strip()!='N/A':
            kid_games.append(row.find_all('a')[1].get('href'))



100%|██████████| 14/14 [00:03<00:00,  3.51it/s]


In [6]:
for elm in map(lambda x: x.split('/')[2], kid_games):
    game_id.append(elm)


In [7]:
family_games = []
for i in tqdm(range(1,16)):

    # Connect to the URL
    response = requests.get(f'https://boardgamegeek.com/familygames/browse/boardgame/page/{i}')

    # Parse HTML and save to BeautifulSoup object¶
    soup = BeautifulSoup(response.text, "html.parser")
    for row in soup.find_all(id='row_'):
        if row.find_all('td')[3].contents[0].strip()!='N/A':
            family_games.append(row.find_all('a')[1].get('href'))


100%|██████████| 15/15 [00:05<00:00,  2.84it/s]


In [8]:
for elm in map(lambda x: x.split('/')[2], family_games):
    game_id.append(elm)


In [9]:
len(set(game_id))

2340

In [10]:
game_id[0]

'174430'

In [93]:
import os
stats_kws = ['usersrated','average', 'bayesaverage', 'stddev', 'median', 'owned', 'trading','wanting','wishing','numcomments', 'numweights','averageweight']
info_kws = ['name', 'yearpublished', 'minplayers', 'maxplayers', 'playingtime','minplaytime','maxplaytime','age']
# See what's been saved
done = set()

#for id in tqdm(set(game_id).difference(map(lambda x: x.split('.')[0],os.listdir('bbg_json')))):

for id in tqdm(set(game_id)):
    game_json = {}
    #First get the stats
    response = requests.get(f'https://www.boardgamegeek.com/xmlapi/boardgame/{id}?stats=1')

    # Parse HTML and save to BeautifulSoup object¶
    soup = BeautifulSoup(response.text, "html.parser")
    for kw in info_kws:
        game_json[kw] = soup.find(kw).contents[0]
    game_json['description'] = soup.find('description').contents[0].replace('<br/>', ' ').strip()
    try:
        game_json['img_url'] = soup.find('thumbnail').contents[0]
    except IndexError:
        game_json['img_url'] = 'N/A'
    game_json['user_suggested_age'] = 0
    game_json['user_suggested_players'] = []

    for poll in soup.find_all('poll'):

        if poll.get('name') == "suggested_playerage":
            total_votes = float(poll.get('totalvotes'))
            if total_votes > 0:
                for elm in poll.find_all('result'):
                    game_json['user_suggested_age'] += float(elm.get('value').split(' ')[0])*float(elm.get('numvotes'))
                game_json['user_suggested_age'] /= total_votes
        elif poll.get('name') == 'suggested_numplayers':
            for results in poll.find_all('results'):
                suggested_num_dict = {'player_num' : results.get('numplayers')}
                for elm in results.find_all('result'):
                    suggested_num_dict[elm.get('value')] = elm.get('numvotes')
                game_json['user_suggested_players'].append(suggested_num_dict)
            
    stats = soup.find('ratings')
    game_json['stats'] = {}
    for kw in stats_kws:
        game_json['stats'][kw] = float(stats.find(kw).contents[0])
    num_comments = min(game_json['stats']['numcomments'],100)
    
                
    game_json['comments'] = []


    j = 1

    while (num_comments) > 0:
        response = requests.get(f'https://www.boardgamegeek.com/xmlapi/boardgame/{id}?comments=j')

        # Parse HTML and save to BeautifulSoup object¶
        soup = BeautifulSoup(response.text, "html.parser")
       
        
        for elm in soup.find_all('comment'):
            comment_dict = {}
            comment_dict['rating'] = elm.get('rating')
            comment_dict['username'] = elm.get('username')
            comment_dict['comment'] = elm.contents[0]
            game_json['comments'].append(comment_dict)
        j += 1
        num_comments -=50

    with open(f'bbg_json/{id}.json', 'w') as f:
        json.dump(game_json, f)





  0%|          | 0/534 [00:00<?, ?it/s]


  0%|          | 1/534 [00:00<02:47,  3.19it/s]


  0%|          | 2/534 [00:00<03:12,  2.77it/s]


  1%|          | 3/534 [00:01<03:37,  2.44it/s]


  1%|          | 4/534 [00:02<05:03,  1.75it/s]


  1%|          | 5/534 [00:02<04:58,  1.77it/s]


  1%|          | 6/534 [00:03<04:47,  1.84it/s]


  1%|▏         | 7/534 [00:03<04:50,  1.81it/s]


  1%|▏         | 8/534 [00:04<04:36,  1.90it/s]


  2%|▏         | 9/534 [00:04<04:32,  1.93it/s]


  2%|▏         | 10/534 [00:05<04:25,  1.97it/s]


  2%|▏         | 11/534 [00:05<04:23,  1.98it/s]


  2%|▏         | 12/534 [00:07<06:51,  1.27it/s]


  2%|▏         | 13/534 [00:08<07:54,  1.10it/s]


  3%|▎         | 14/534 [00:09<08:39,  1.00it/s]


  3%|▎         | 15/534 [00:10<07:50,  1.10it/s]


  3%|▎         | 16/534 [00:11<07:19,  1.18it/s]


  3%|▎         | 17/534 [00:11<06:30,  1.32it/s]


  3%|▎         | 18/534 [00:12<05:48,  1.48it/s]


  4%|▎         | 19/534 [00:12<05:02,  1.70it/

 25%|██▌       | 136/534 [01:12<02:45,  2.41it/s]


 26%|██▌       | 137/534 [01:13<02:50,  2.33it/s]


 26%|██▌       | 138/534 [01:13<02:52,  2.29it/s]


 26%|██▌       | 139/534 [01:14<03:10,  2.07it/s]


 26%|██▌       | 140/534 [01:14<03:10,  2.07it/s]


 26%|██▋       | 141/534 [01:15<03:18,  1.98it/s]


 27%|██▋       | 142/534 [01:15<03:12,  2.04it/s]


 27%|██▋       | 143/534 [01:16<03:14,  2.02it/s]


 27%|██▋       | 144/534 [01:16<03:11,  2.04it/s]


 27%|██▋       | 145/534 [01:17<03:45,  1.73it/s]


 27%|██▋       | 146/534 [01:18<03:17,  1.97it/s]


 28%|██▊       | 147/534 [01:18<04:11,  1.54it/s]


 28%|██▊       | 148/534 [01:19<03:30,  1.83it/s]


 28%|██▊       | 149/534 [01:19<03:22,  1.90it/s]


 28%|██▊       | 150/534 [01:20<03:13,  1.99it/s]


 28%|██▊       | 151/534 [01:20<02:53,  2.21it/s]


 28%|██▊       | 152/534 [01:21<03:00,  2.12it/s]


 29%|██▊       | 153/534 [01:21<03:38,  1.75it/s]


 29%|██▉       | 154/534 [01:22<03:29,  1.81it/s]


 29%|██▉    

 51%|█████     | 270/534 [02:24<02:08,  2.06it/s]


 51%|█████     | 271/534 [02:24<02:08,  2.05it/s]


 51%|█████     | 272/534 [02:25<02:09,  2.02it/s]


 51%|█████     | 273/534 [02:25<02:11,  1.98it/s]


 51%|█████▏    | 274/534 [02:26<02:13,  1.95it/s]


 51%|█████▏    | 275/534 [02:26<02:13,  1.94it/s]


 52%|█████▏    | 276/534 [02:27<02:11,  1.96it/s]


 52%|█████▏    | 277/534 [02:27<02:08,  2.00it/s]


 52%|█████▏    | 278/534 [02:28<02:15,  1.89it/s]


 52%|█████▏    | 279/534 [02:28<02:17,  1.86it/s]


 52%|█████▏    | 280/534 [02:29<02:18,  1.83it/s]


 53%|█████▎    | 281/534 [02:30<02:12,  1.91it/s]


 53%|█████▎    | 282/534 [02:30<02:12,  1.91it/s]


 53%|█████▎    | 283/534 [02:31<02:12,  1.89it/s]


 53%|█████▎    | 284/534 [02:31<02:07,  1.96it/s]


 53%|█████▎    | 285/534 [02:32<03:03,  1.36it/s]


 54%|█████▎    | 286/534 [02:33<02:35,  1.59it/s]


 54%|█████▎    | 287/534 [02:33<02:15,  1.82it/s]


 54%|█████▍    | 288/534 [02:34<02:19,  1.76it/s]


 54%|█████▍ 

 76%|███████▌  | 404/534 [03:33<01:10,  1.84it/s]


 76%|███████▌  | 405/534 [03:33<01:07,  1.92it/s]


 76%|███████▌  | 406/534 [03:34<01:07,  1.90it/s]


 76%|███████▌  | 407/534 [03:34<01:02,  2.04it/s]


 76%|███████▋  | 408/534 [03:35<01:02,  2.01it/s]


 77%|███████▋  | 409/534 [03:35<00:59,  2.09it/s]


 77%|███████▋  | 410/534 [03:36<01:05,  1.90it/s]


 77%|███████▋  | 411/534 [03:36<01:04,  1.91it/s]


 77%|███████▋  | 412/534 [03:37<01:05,  1.87it/s]


 77%|███████▋  | 413/534 [03:37<01:05,  1.86it/s]


 78%|███████▊  | 414/534 [03:38<00:57,  2.09it/s]


 78%|███████▊  | 415/534 [03:38<00:57,  2.06it/s]


 78%|███████▊  | 416/534 [03:39<00:56,  2.08it/s]


 78%|███████▊  | 417/534 [03:39<00:56,  2.09it/s]


 78%|███████▊  | 418/534 [03:40<00:56,  2.06it/s]


 78%|███████▊  | 419/534 [03:41<01:13,  1.56it/s]


 79%|███████▊  | 420/534 [03:41<01:07,  1.70it/s]


 79%|███████▉  | 421/534 [03:42<01:03,  1.79it/s]


 79%|███████▉  | 422/534 [03:42<01:08,  1.63it/s]


 79%|███████

In [98]:
len(os.listdir('bbg_json')), len(set(game_id))

(2340, 2340)